In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
f = open('car.data', 'r')
car = pd.read_csv(f)
sf = np.random.permutation(car.shape[0])
car = car.iloc[sf]

In [4]:
train_row = range(car.shape[0] / 3 * 2)
test_row = range(car.shape[0] / 3 * 2, car.shape[0])

train_car = car.iloc[train_row]
test_car = car.iloc[test_row]

In [5]:
train_car.shape

(1152, 7)

In [6]:
def HA(D, A):
    n = D.shape[0]
    values = D[A].unique()
    DI = np.array([split(D, A, value).shape[0] for value in values])
    P = DI / float(n)
    return -np.sum(P * np.log(P))

def entropy(D):
    return HA(D, 'class')

def H(D, a):
    ent = [entropy(split(D, a, value)) for value in D[a].unique()]
    size = [split(D, a, value).shape[0] for value in D[a].unique()]
    return (np.array(ent) * np.array(size) / D.shape[0]).sum()

def g(D, a):
    return entropy(D) - H(D, a)

def delta_ent_rate(D, a):
    return g(D, a) / HA(D, a)

def split(data, A, value):
    return data[data[A] == value]

def maxCntFeat(D):
    label = D['class'].unique()
    mLableIndex = np.argmax([D[D['class'] == lbl].shape[0] for lbl in label])
    return label[mLableIndex]

def createTree(data, features, eps=0.16):
    n = data.shape[0]
    cls = data['class']
    if cls[cls == cls.iloc[0]].size == n:
        return maxCntFeat(data), n
    if features.size == 0:
        return maxCntFeat(data), n

    bestFeat = pd.Series({f: delta_ent_rate(data, f) for f in features}).argmax()
    tree = {bestFeat : {}}
    restFeat = np.delete(features, np.where(features == bestFeat)[0][0])
    #print restFeat
    
    for value in car[bestFeat].unique():
        Dv = split(data, bestFeat, value)
        if Dv.shape[0] == 0:
            tree[bestFeat][value] = maxCntFeat(data), 0
        else:
            tree[bestFeat][value] = createTree(split(data, bestFeat, value), restFeat)
        
    return tree



In [7]:
def classfy(tree, sample):
    while True:
        if type(tree) == tuple:
            return tree[0]
        feat = tree.keys()[0]
        sampleFeat = sample[feat]
        tree = tree[feat][sampleFeat]
        
def predict(tree, sample):
    return np.array([classfy(tree, sample.iloc[i]) for i in range(sample.shape[0])])

In [8]:
tree = createTree(train_car, np.array(car.columns[:-1]))
output = predict(tree, test_car)
np.sum(test_car['class'].values == output) / float(output.shape[0])


0.91840277777777779

In [9]:
output.shape

(576L,)

In [10]:
car['safety'].unique()

array(['low', 'high', 'med'], dtype=object)